# Sandbox notebook

### Importing Libraries

In [1]:
import gc
import os
import sys
import math
import glob
import tqdm
import random
import numpy as np
from tqdm import tqdm
from time import sleep

In [2]:
import pandas as pd
import xml.etree.cElementTree as ET

In [3]:
import cv2
import dlib
from imutils import face_utils
from skimage.feature import hog
from skimage import data,exposure

### Defining Paths

In [4]:
path_openface_data = '/media/amogh/Stuff/oface'

In [5]:
!ls output_XML/

18235.xml
cnn_output.xml
dlib_hog_output_232_folders.xml
hog_output230.xml
hog_output231.xml
initial_openface.xml
openface_166_folders_75_confidence.xml
openface_166_folders_85_confidence.xml
openface_166_folders.xml
openface_cnn_168_folders_75_confidence.xml
openface_cnn_168_folders_85_confidence.xml
openface_dlib_168_folders_75_confidence.xml
openface_dlib_168_folders_85_confidence.xml
zface-output.xml


## Aggregating OpenFace csv to xml

In [6]:
def getXMLFromOpenFaceData(path_folder_openface_data, conf, path_output_xml = 'output_XML/'):
    """
    Generates XML data file from a folder containing the output csv files from OpenFace.
    """
    
    list_csv_files = glob.glob(path_folder_openface_data + '/*.csv')
    
    #Generate XML file.
    root = ET.Element("dataset")
    name = ET.SubElement(root, "name").text = "Labelled faces"
    comment = ET.SubElement(root, "comment").text = "These are labelled images from Bagamoyo"
    images = ET.SubElement(root, "images")
    # Read csv file and populate the XML data.
    for path_csv in tqdm(list_csv_files):
        name_video = os.path.splitext(os.path.basename(path_csv))[0]
        df_data = pd.read_csv(path_csv)
        
        # Iterate over each frame in this video's output
        for index, row in df_data.iterrows():
            #define the image node for which box will be a child.
            frame_num = row.frame
            image_node_file_name = name_video + " " + "%03d"%frame_num + ".jpg"
            confidence = row[" confidence"]
#             print(confidence)
            image_node=None
            #check if the attribute for frame already exists, create if not.
            for image_elem in images.findall('image'):
                file_name = image_elem.get('file')
                if(file_name == image_node_file_name):
                    image_node = image_elem
                    print("found")
            if(image_node is None):
                image_node = ET.SubElement(images, "image", file=image_node_file_name)
                
            #get box coordinates
            list_x_coord_landmarks = list(row[' x_0':' x_67']) 
            list_y_coord_landmarks = list(row[' y_0':' y_67'])
            left = int(min(list_x_coord_landmarks))
            right = int(max(list_x_coord_landmarks))
            top = int(min(list_y_coord_landmarks))
            bottom = int(max(list_y_coord_landmarks))
            width = right - left
            height = bottom - top
            #add box node
            if(confidence > conf):
                box = ET.SubElement(image_node, "box", height=str(height), left=str(left), top=str(top), width=str(width))
    
    #save the XML
    tree = ET.ElementTree(root)
    dest_name = path_output_xml + 'openface_{}_folders_{}_confidence.xml'.format(len(list_csv_files),int(100*conf))
    tree.write(dest_name)
    print("xml written to {}".format(dest_name))  

In [8]:
getXMLFromOpenFaceData('/media/amogh/Stuff/CMU/datasets/bagamoyo_data/openface_outputs/library/',0.75,'/media/amogh/Stuff/CMU/datasets/bagamoyo_data/openface_outputs/library/')  

100%|██████████| 199/199 [01:29<00:00,  2.22it/s]


xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/openface_outputs/library/openface_199_folders_75_confidence.xml


In [ ]:
getXMLFromOpenFaceData('try')

In [13]:
path_openface_cnn_data = "/media/amogh/Stuff/OpenFace_Outputs/Openface_outputs/new_oface_cnn_detector/"
getXMLFromOpenFaceData(path_openface_cnn_data,0.85)

100%|██████████| 168/168 [01:13<00:00,  2.29it/s]


xml written to output_XML/openface_168_folders_85_confidence.xml


In [14]:
path_openface_dlib_data = "/media/amogh/Stuff/OpenFace_Outputs/Openface_outputs/new_oface_dlib_detector/"
getXMLFromOpenFaceData(path_openface_dlib_data,0.85)

100%|██████████| 168/168 [01:10<00:00,  2.38it/s]


xml written to output_XML/openface_168_folders_85_confidence.xml


## Debug

In [34]:
def debug():
    root = ET.Element("dataset")
    name = ET.SubElement(root, "name", file='a')
#     name = ET.SubElement(root, "name", file='a')
    name = ET.SubElement(root, "name", file='a')
    print([a for a in root])
    tree = ET.ElementTree(root)
    tree.write('try.xml')
debug()

[<Element 'name' at 0x7f001761a720>, <Element 'name' at 0x7f001761a7e0>]
